# Chuck Tingle Title Generator

This notebook will generate Chuck Tingle titles using Tensorflow and LSTM. We will give the model an initial word or words, and the total number of words expected in the title. The LSTM will then generate the next word(s).


Inspired by:
https://towardsdatascience.com/simple-text-generation-using-lstm-deep-learning-d9ba808905ff

In [1]:
# load packages.
import tensorflow as tf
tf.enable_eager_execution()
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence

import pandas as pd
import numpy as np
import os
import time


/home/vector/anaconda3/envs/KerasGPU2.2.4/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/vector/anaconda3/envs/KerasGPU2.2.4/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/vector/anaconda3/envs/KerasGPU2.2.4/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/vector/anaconda3/en

# Preprocessing

In [120]:
class Preprocessing():
    def __init__(self, input_data_file):
        self.data = None
        self.x = None
        self.y = None
        self.tokenizer = None
        self.encoded_data = None
        self.num_unique_words = None
        self.max_len = None
        self.sequences = None
        self.num_data = None
        self.input_data_file = input_data_file
            
    def load(self, column):
        # reads in a file using pandas. Selects a column to be data
        dataframe = pd.read_csv(self.input_data_file, error_bad_lines=False)
        self.data = dataframe[column].values
        
    def encode(self):
        # tokenizes the data
        self.tokenizer = Tokenizer()
        self.tokenizer.fit_on_texts(self.data)
        self.encoded_data = self.tokenizer.texts_to_sequences(self.data)
        self.num_unique_words = len(self.tokenizer.word_counts)+1
        self.max_len = max([len(data) for data in self.encoded_data])

    def generate_sequence(self):
        # get list of outputs
        seq_list = list()
        for data_point in self.encoded_data:
            for index in range(1, len(data_point)):
                seq_list.append(data_point[:index+1])
        # pad sequences
        self.sequences = np.array(sequence.pad_sequences(seq_list, maxlen = self.max_len))    
        self.num_data = len(self.sequences)
        
    def get_xy(self):
        self.x = self.sequences[:,:-1]
        # set the last word of the sequence as the y data
        self.y = self.sequences[:,-1]
        self.y = tf.keras.utils.to_categorical(self.sequences[:,-1], num_classes=self.num_unique_words)
    



In [122]:
# Load file containing all book titles.

data = Preprocessing('BookList.txt')
data.load('title')
data.encode()
data.generate_sequence()
data.get_xy()


# Build Model

In [170]:
# build model
class LSTM_model():
    def __init__(self, params):
        self.model = None
        self.history = None
        self.num_unique_words = params['num_unique_words']
        self.max_len = params['max_len']
        self.activation = params['activation']
        self.optimizer = params['optimizer']
        self.epochs = params['epochs']
        self.metrics = params['metrics']
        self.loss = params['loss']
        self.batch_size = params['batch_size']
        self.shuffle = params['shuffle']
    
    def built_model(self):
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.Embedding(self.num_unique_words, 16,input_length=self.max_len-1))
        self.model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))
        self.model.add(tf.keras.layers.Dropout(0.2))
        self.model.add(tf.keras.layers.Dense(self.num_unique_words,activation=self.activation))
        self.model.compile(loss=self.loss,optimizer=self.optimizer,metrics=self.metrics)

    def train(self):
        self.history = self.model.fit(self.x, self.y,
                                      epochs = self.epochs,
                                      shuffle = self.shuffle,
                                      batch_size=self.batch_size)


In [173]:
params = {"activation":"softmax",
          "epochs":700,
          "loss":"categorical_crossentropy",
          "optimizer":tf.keras.optimizers.Adam(lr=1e-3),
          "metrics":["accuracy"],
          "num_unique_words":data.num_unique_words,
          "max_len":data.max_len,
          "batch_size":512,
          "shuffle":True}
model_obj = LSTM_model(params)
model_obj.x = data.x
model_obj.y = data.y
model_obj.built_model()

In [174]:
# train
model_obj.train()

Epoch 1/700
2270/2270 [==============================] - 4s 2ms/sample - loss: 6.7390 - acc: 0.0330
Epoch 2/700
2270/2270 [==============================] - 1s 259us/sample - loss: 6.7215 - acc: 0.0744
Epoch 3/700
2270/2270 [==============================] - 1s 230us/sample - loss: 6.6802 - acc: 0.0775
Epoch 4/700
2270/2270 [==============================] - 1s 228us/sample - loss: 6.4988 - acc: 0.0780
Epoch 5/700
2270/2270 [==============================] - 1s 325us/sample - loss: 5.9039 - acc: 0.0700
Epoch 6/700
2270/2270 [==============================] - 1s 272us/sample - loss: 5.5005 - acc: 0.0705
Epoch 7/700
2270/2270 [==============================] - 1s 226us/sample - loss: 5.3876 - acc: 0.0793
Epoch 8/700
2270/2270 [==============================] - 1s 227us/sample - loss: 5.3849 - acc: 0.0780
Epoch 9/700
2270/2270 [==============================] - 1s 254us/sample - loss: 5.3666 - acc: 0.0797
Epoch 10/700
2270/2270 [==============================] - 1s 229us/sample - loss: 5.

2270/2270 [==============================] - 1s 240us/sample - loss: 4.4569 - acc: 0.1722
Epoch 81/700
2270/2270 [==============================] - 1s 375us/sample - loss: 4.4447 - acc: 0.1863
Epoch 82/700
2270/2270 [==============================] - 0s 217us/sample - loss: 4.4273 - acc: 0.1846
Epoch 83/700
2270/2270 [==============================] - 1s 228us/sample - loss: 4.4213 - acc: 0.1846
Epoch 84/700
2270/2270 [==============================] - 1s 223us/sample - loss: 4.3950 - acc: 0.1877
Epoch 85/700
2270/2270 [==============================] - 1s 257us/sample - loss: 4.3825 - acc: 0.1899
Epoch 86/700
2270/2270 [==============================] - 1s 229us/sample - loss: 4.3705 - acc: 0.1903
Epoch 87/700
2270/2270 [==============================] - 1s 351us/sample - loss: 4.3483 - acc: 0.1987
Epoch 88/700
2270/2270 [==============================] - 1s 277us/sample - loss: 4.3325 - acc: 0.1947
Epoch 89/700
2270/2270 [==============================] - 1s 232us/sample - loss: 4.30

2270/2270 [==============================] - 1s 225us/sample - loss: 3.5312 - acc: 0.3119
Epoch 160/700
2270/2270 [==============================] - 1s 222us/sample - loss: 3.5142 - acc: 0.3150
Epoch 161/700
2270/2270 [==============================] - 1s 227us/sample - loss: 3.4997 - acc: 0.3057
Epoch 162/700
2270/2270 [==============================] - 1s 261us/sample - loss: 3.4751 - acc: 0.3115
Epoch 163/700
2270/2270 [==============================] - 1s 241us/sample - loss: 3.4482 - acc: 0.3172
Epoch 164/700
2270/2270 [==============================] - 1s 249us/sample - loss: 3.4413 - acc: 0.3154
Epoch 165/700
2270/2270 [==============================] - 1s 390us/sample - loss: 3.4192 - acc: 0.3159
Epoch 166/700
2270/2270 [==============================] - 1s 245us/sample - loss: 3.4038 - acc: 0.3137
Epoch 167/700
2270/2270 [==============================] - 1s 230us/sample - loss: 3.3851 - acc: 0.3150
Epoch 168/700
2270/2270 [==============================] - 1s 234us/sample - l

2270/2270 [==============================] - 1s 373us/sample - loss: 2.6369 - acc: 0.4189
Epoch 238/700
2270/2270 [==============================] - 1s 254us/sample - loss: 2.6295 - acc: 0.4251
Epoch 239/700
2270/2270 [==============================] - 1s 248us/sample - loss: 2.6164 - acc: 0.4159
Epoch 240/700
2270/2270 [==============================] - 1s 249us/sample - loss: 2.6075 - acc: 0.4256
Epoch 241/700
2270/2270 [==============================] - 1s 228us/sample - loss: 2.5764 - acc: 0.4330
Epoch 242/700
2270/2270 [==============================] - 0s 220us/sample - loss: 2.5877 - acc: 0.4220
Epoch 243/700
2270/2270 [==============================] - 1s 248us/sample - loss: 2.5727 - acc: 0.4317
Epoch 244/700
2270/2270 [==============================] - 1s 239us/sample - loss: 2.5638 - acc: 0.4410
Epoch 245/700
2270/2270 [==============================] - 1s 221us/sample - loss: 2.5499 - acc: 0.4300
Epoch 246/700
2270/2270 [==============================] - 1s 248us/sample - l

2270/2270 [==============================] - 1s 277us/sample - loss: 1.9988 - acc: 0.5705
Epoch 316/700
2270/2270 [==============================] - 1s 304us/sample - loss: 1.9972 - acc: 0.5740
Epoch 317/700
2270/2270 [==============================] - 0s 217us/sample - loss: 1.9827 - acc: 0.5705
Epoch 318/700
2270/2270 [==============================] - 1s 222us/sample - loss: 1.9710 - acc: 0.5811
Epoch 319/700
2270/2270 [==============================] - 1s 225us/sample - loss: 1.9704 - acc: 0.5709
Epoch 320/700
2270/2270 [==============================] - 1s 223us/sample - loss: 1.9657 - acc: 0.5696
Epoch 321/700
2270/2270 [==============================] - 1s 229us/sample - loss: 1.9654 - acc: 0.5811
Epoch 322/700
2270/2270 [==============================] - 1s 243us/sample - loss: 1.9420 - acc: 0.5921
Epoch 323/700
2270/2270 [==============================] - 1s 224us/sample - loss: 1.9415 - acc: 0.5828
Epoch 324/700
2270/2270 [==============================] - 1s 221us/sample - l

2270/2270 [==============================] - 0s 213us/sample - loss: 1.5268 - acc: 0.6828
Epoch 394/700
2270/2270 [==============================] - 1s 222us/sample - loss: 1.5116 - acc: 0.6855
Epoch 395/700
2270/2270 [==============================] - 0s 208us/sample - loss: 1.5185 - acc: 0.6855
Epoch 396/700
2270/2270 [==============================] - 0s 205us/sample - loss: 1.5036 - acc: 0.6872
Epoch 397/700
2270/2270 [==============================] - 0s 209us/sample - loss: 1.5057 - acc: 0.6903
Epoch 398/700
2270/2270 [==============================] - 1s 223us/sample - loss: 1.4952 - acc: 0.6868
Epoch 399/700
2270/2270 [==============================] - 1s 226us/sample - loss: 1.4859 - acc: 0.6969
Epoch 400/700
2270/2270 [==============================] - 1s 236us/sample - loss: 1.4879 - acc: 0.6930
Epoch 401/700
2270/2270 [==============================] - 1s 255us/sample - loss: 1.4738 - acc: 0.6833
Epoch 402/700
2270/2270 [==============================] - 1s 363us/sample - l

2270/2270 [==============================] - 1s 225us/sample - loss: 1.1883 - acc: 0.7502
Epoch 472/700
2270/2270 [==============================] - 1s 323us/sample - loss: 1.1700 - acc: 0.7493
Epoch 473/700
2270/2270 [==============================] - 1s 266us/sample - loss: 1.1719 - acc: 0.7480
Epoch 474/700
2270/2270 [==============================] - 0s 218us/sample - loss: 1.1788 - acc: 0.7419
Epoch 475/700
2270/2270 [==============================] - 0s 219us/sample - loss: 1.1523 - acc: 0.7608
Epoch 476/700
2270/2270 [==============================] - 0s 217us/sample - loss: 1.1490 - acc: 0.7515
Epoch 477/700
2270/2270 [==============================] - 1s 252us/sample - loss: 1.1752 - acc: 0.7480
Epoch 478/700
2270/2270 [==============================] - 1s 226us/sample - loss: 1.1697 - acc: 0.7604
Epoch 479/700
2270/2270 [==============================] - 1s 231us/sample - loss: 1.1586 - acc: 0.7573
Epoch 480/700
2270/2270 [==============================] - 1s 302us/sample - l

2270/2270 [==============================] - 1s 232us/sample - loss: 0.9397 - acc: 0.7965
Epoch 550/700
2270/2270 [==============================] - 1s 227us/sample - loss: 0.9265 - acc: 0.8009
Epoch 551/700
2270/2270 [==============================] - 1s 266us/sample - loss: 0.9195 - acc: 0.8018
Epoch 552/700
2270/2270 [==============================] - 0s 220us/sample - loss: 0.9295 - acc: 0.7991
Epoch 553/700
2270/2270 [==============================] - 1s 234us/sample - loss: 0.9237 - acc: 0.8018
Epoch 554/700
2270/2270 [==============================] - 1s 318us/sample - loss: 0.9216 - acc: 0.8009
Epoch 555/700
2270/2270 [==============================] - 1s 306us/sample - loss: 0.9213 - acc: 0.7987
Epoch 556/700
2270/2270 [==============================] - 1s 233us/sample - loss: 0.9135 - acc: 0.8013
Epoch 557/700
2270/2270 [==============================] - 1s 233us/sample - loss: 0.9112 - acc: 0.8053
Epoch 558/700
2270/2270 [==============================] - 1s 233us/sample - l

2270/2270 [==============================] - 0s 213us/sample - loss: 0.7726 - acc: 0.8229
Epoch 628/700
2270/2270 [==============================] - 0s 204us/sample - loss: 0.7654 - acc: 0.8295
Epoch 629/700
2270/2270 [==============================] - 0s 207us/sample - loss: 0.7573 - acc: 0.8313
Epoch 630/700
2270/2270 [==============================] - 0s 209us/sample - loss: 0.7664 - acc: 0.8264
Epoch 631/700
2270/2270 [==============================] - 1s 259us/sample - loss: 0.7582 - acc: 0.8344
Epoch 632/700
2270/2270 [==============================] - 0s 209us/sample - loss: 0.7669 - acc: 0.8264
Epoch 633/700
2270/2270 [==============================] - 0s 206us/sample - loss: 0.7584 - acc: 0.8251
Epoch 634/700
2270/2270 [==============================] - 1s 246us/sample - loss: 0.7638 - acc: 0.8295
Epoch 635/700
2270/2270 [==============================] - 1s 354us/sample - loss: 0.7937 - acc: 0.8264
Epoch 636/700
2270/2270 [==============================] - 1s 221us/sample - l

# Predictions

In [182]:
class Prediction():
    def __init__(self,tokenizer,max_len):
        self.model = None
        self.tokenizer = tokenizer
        self.max_length = max_len
    
    def load_model(self):
        self.model = tf.keras.models.load_model("lang_model.h5")

    def predict_sequnce(self, text, num_words_gen):
        for id in range(num_words_gen):
            # convert input text to tokenized and padded sequence
            encoded_data = self.tokenizer.texts_to_sequences([text])[0]
            padded_data = tf.keras.preprocessing.sequence.pad_sequences([encoded_data],maxlen = self.max_length-1,padding='pre')
            # predict next word
            y_pred = self.model.predict(padded_data)
            y_pred = np.argmax(y_pred)
            # convert prediction to word using index_word in tokenizer
            predict_word = self.tokenizer.index_word[y_pred]
            # append predicted word to existing text
            text += ' ' + predict_word
        return text
    

pred = Prediction(data.tokenizer, data.max_len)  
pred.model = model_obj.model
print(pred.predict_sequnce("Jack", 11))
print(pred.predict_sequnce('Shrek and', 12))
print(pred.predict_sequnce('Mothman', 10))
print(pred.predict_sequnce('My', 8))    

Jack last missing piece for the jigsaw puzzle i've been working on
Shrek and man pounded by the handsome physically manifested sound that some people hear
Mothman wizard harriet also a lesbian and she eats me right
My librarian is a beautiful lesbian ice cream cone
